# Task 3. Algorithms of unconditional nonlinear optimization. First and second order methods

### Generate random numbers $\alpha \in (0, 1)$  and $\beta \in (0, 1)$. Furthermore, generate the noisy data {$x_k, y_k$}, where $k = 0, 1, .. 100$, according to the rule: $x_k = \frac{k}{100}$, ${y_k = \alpha x_k + \beta + \delta_k}$, where $\delta_k \sim N(0, 1)$ are values of a random variable with standard normal distribution. Approximate the data by the following linear and rational functions:

### $1) F(x, a, b) = ax + b$ (linear approximant)
### $2) F(x, a, b) = \frac{a}{1 + bx}$ (rational approximant)

### by means of least squares through the numerical minimization (with precision $\varepsilon = 0.001$) of the following function:

### $D(a, b) = \sum\limits_{k = 0}^{100}{(F(x_k, a, b) - y_k)^2}$

### To solve the minimization problem, use gradient descent,the conjugate gradient method, the Newton method and the Levenberg-Marquardt algorithm. If necessary, set the initial approximations and other parameters of the methods yourself. 

### On the graph (separately for each approximating function), draw an array of generateddata and graphs of approximating functions obtained using these numerical optimization algorithms. Analyzethe results obtained in terms of the number of iterations performed. Compare the results obtained with the results from previous task


In [88]:
import numpy as np
import pandas as pd
from scipy import misc
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import minimize, minimize_scalar

global x_arr
global y_arr


def least_squares_linear(args):
    return np.sum(np.square(args[0] * x_arr + args[1] - y_arr))


def least_squares_rational(args):
    return np.sum(np.square(args[0] / (1 + args[1] * x_arr) - y_arr))


def gradient_scipy(func, args, eps = 1e-5):
    
    a_arg, b_arg = args[0], args[1]

    part_deriv_a = misc.derivative(lambda a: func([a, b_arg]), a_arg, dx = eps)
    part_deriv_b = misc.derivative(lambda b: func([a_arg, b]), b_arg, dx = eps)

    return np.array(part_deriv_a, part_deriv_b)


def gradient_num(func, args, eps = 1e-5):
    
    a_arg, b_arg = args[0], args[1]
    
    #n_digits = abs(int(round(np.log10(eps))))
    #part_deriv_a = round((func([a_arg + eps, b_arg]) - func([a_arg - eps, b_arg])) / (2 * eps), n_digits)
    #part_deriv_b = round((func([a_arg, b_arg + eps]) - func([a_arg, b_arg - eps])) / (2 * eps), n_digits)
    
    part_deriv_a = (func([a_arg + eps, b_arg]) - func([a_arg - eps, b_arg])) / (2 * eps)
    part_deriv_b = (func([a_arg, b_arg + eps]) - func([a_arg, b_arg - eps])) / (2 * eps)

    return np.array([part_deriv_a,  part_deriv_b])


def hessian_num(func, args, eps = 1e-5):                            # value of hessian of func at the point args = [a, b]
    
    a_arg, b_arg = args[0], args[1]
    
    func_ab = func([a_arg, b_arg])
    func_a_eps_b = func([a_arg + eps, b_arg])
    func_a_b_eps = func([a_arg, b_arg + eps])
    
    #n_digits = abs(int(round(np.log10(eps))))
    #part_deriv_aa = round((func_a_eps_b - 2 * func_ab + func([a_arg - eps, b_arg])) / (eps ** 2), n_digits)
    #part_deriv_bb = round((func_a_b_eps - 2 * func_ab + func([a_arg, b_arg - eps])) / (eps ** 2), n_digits)
    #part_deriv_ab = round((func([a_arg + eps, b_arg + eps]) - func_a_eps_b - func_a_b_eps + func_ab) / (eps ** 2), n_digits)

    part_deriv_aa = (func_a_eps_b - 2 * func_ab + func([a_arg - eps, b_arg])) / (eps ** 2)
    part_deriv_bb = (func_a_b_eps - 2 * func_ab + func([a_arg, b_arg - eps])) / (eps ** 2)
    part_deriv_ab = (func([a_arg + eps, b_arg + eps]) - func_a_eps_b - func_a_b_eps + func_ab) / (eps ** 2)
    
    return np.array([[part_deriv_aa, part_deriv_ab], [part_deriv_ab, part_deriv_bb]])


def gradient_descent(func, initial_approx = [1, 0], eps = 1e-4):
    
    func_eval_num, iter_num = 0, 0
    args_prev = np.array(initial_approx)
    
    args_curr = args_prev - np.dot(gradient_num(func, args_prev), 0.01)
    #grad_func_curr = gradient_num(func, args_curr)
    
    func_eval_num += 4
    iter_num += 1
    
    while (np.linalg.norm(args_curr - args_prev, np.inf) >= eps) and (iter_num < 1000):
        
        #grad_func_prev = grad_func_curr[:]
        grad_func_prev = gradient_num(func, args_prev)
        grad_func_curr = gradient_num(func, args_curr)
        
        beta = np.dot(args_curr - args_prev, grad_func_curr - grad_func_prev) \
        / np.square(np.linalg.norm(grad_func_curr - grad_func_prev))
        
        #args_prev = args_curr[:]
        args_prev = args_curr
        args_curr = args_prev - np.dot(gradient_num(func, args_prev), beta)
        
        #cond = abs(func(args_curr) - func(args_prev))
        #print(args_curr[0], args_curr[1], cond) 
        #plt.scatter(args_curr[0], args_curr[1])
        
        func_eval_num +=  12
        iter_num += 1
    
    n_digits = abs(int(round(np.log10(eps))))

    return np.array([round(args_curr[0], n_digits), round(args_curr[1], n_digits), int(func_eval_num), int(iter_num)])


def conjugate_gradient_descent(func, initial_approx = [1, 0], eps = 1e-4):
    
    func_eval_num, iter_num = 0, 0

    args_prev = np.array(initial_approx)
    grad_func_prev = gradient_num(func, args_prev)
    res = minimize_scalar(lambda x: func(args_prev - x * grad_func_prev))
    
    args_curr = args_prev - np.dot(res.x, grad_func_prev)
    conj_direction_curr = -1 * grad_func_prev
    
    #grad_func_curr = gradient_num(func, args_curr)

    func_eval_num += res.nfev + 8
    iter_num += 1
    
    while (np.linalg.norm(args_curr - args_prev, np.inf) >= eps) and (iter_num < 1000):
            
        #grad_func_prev = grad_func_curr[:]
        grad_func_prev = gradient_num(func, args_prev)
        grad_func_curr = gradient_num(func, args_curr)
        
        beta = np.dot(grad_func_curr, grad_func_curr) / np.dot(grad_func_prev, grad_func_prev)
        #beta = np.dot(grad_func_curr, grad_func_curr - grad_func_prev) / np.dot(grad_func_prev, grad_func_prev)
        
        conj_direction_prev = conj_direction_curr[:]
        conj_direction_curr = np.dot(beta, conj_direction_prev) - grad_func_prev
        
        res = minimize_scalar(lambda x: func(args_curr + x * conj_direction_curr))
        
        args_prev = args_curr[:]
        args_curr = args_prev - np.dot(res.x, conj_direction_curr)
        
        #cond = abs(func(args_curr) - func(args_prev))
        #print(args_curr[0], args_curr[1], cond) 
        #plt.scatter(args_curr[0], args_curr[1])
        
        func_eval_num += res.nfev + 4
        iter_num += 1
    
    n_digits = abs(int(round(np.log10(eps))))

    return np.array([round(args_curr[0], n_digits), round(args_curr[1], n_digits), int(func_eval_num), int(iter_num)])


def Newtons_method(func, initial_approx = [1, 0], eps = 1e-4):
    
    res = minimize(func, initial_approx, method = 'Newton-CG', jac = (lambda x: gradient_num(func, x)), \
                   hess = (lambda x: hessian_num(func, x)), options = {'xtol': eps})
    n_digits = abs(int(round(np.log10(eps))))

    return np.array([round(res.x[0], n_digits), round(res.x[1], n_digits), res.nfev, res.nit])
                   
                   
def Levenberg_Marquardt(func, initial_aprox = [1, 0], eps = 1e-4):
    return np.ones(4)


# Pushing in DataFrame results from this and previous laboratory works

x_arr = np.load('x_arr.npy')
y_arr = np.load('y_arr.npy')

df_lin = pd.read_csv('Linear_approx_lab_2.csv', index_col = 0)
df_rat = pd.read_csv('Rational_approx_lab_2.csv', index_col = 0)

algos_dict = {'gradient descent method': gradient_descent, 'conjugate gradient descent method': conjugate_gradient_descent, \
              "Newton's method": Newtons_method, 'Levenberg-Marquardt alghorithm': Levenberg_Marquardt}

for alg_name in algos_dict.keys():
    
    df_lin[alg_name] = algos_dict[alg_name](least_squares_linear)
    df_rat[alg_name] = algos_dict[alg_name](least_squares_rational)
    
df_lin

,exhaustive search,coordinate descent method,Nelder Mead method,gradient descent method,conjugate gradient descent method,Newton's method,Levenberg-Marquardt alghorithm
a,0.451,0.451,0.451,0.4508,1.2370,0.4508,1.0
b,0.965,0.964,0.965,0.9649,0.5481,0.9649,1.0
nfev,7834402.000,1744.000,104.000,88.0000,31.0000,4.0000,1.0
nit,7834402.000,150.000,53.000,8.0000,2.0000,3.0000,1.0


In [89]:
df_rat

,exhaustive search,coordinate descent method,Nelder Mead method,gradient descent method,conjugate gradient descent method,Newton's method,Levenberg-Marquardt alghorithm
a,1.012,1.011,1.012,1.0118,1.1462,1.0118,1.0
b,-0.285,-0.285,-0.285,-0.2848,-0.1025,-0.2848,1.0
nfev,7834402.000,3528.000,129.000,148.0000,64.0000,9.0000,1.0
nit,7834402.000,229.000,68.000,13.0000,2.0000,8.0000,1.0


In [72]:
a = [i for i in range(5)]
b = [(-1) ** i for i in range(5)]
print(a, b, '\n')

for j in range(1, 5):
    
    b = a[:]
    a = [i + j for i in range(5)]
    
    print(a, b, '\n')

#df_rat

np.dot(a, a) == np.sum(np.square(a))

[0, 1, 2, 3, 4] [1, -1, 1, -1, 1] 

[1, 2, 3, 4, 5] [0, 1, 2, 3, 4] 

[2, 3, 4, 5, 6] [1, 2, 3, 4, 5] 

[3, 4, 5, 6, 7] [2, 3, 4, 5, 6] 

[4, 5, 6, 7, 8] [3, 4, 5, 6, 7] 



True

In [28]:
def least_squares_linear(args):                                 # two-variable func to minimaize
    return np.sum(np.square(args[0] * x_arr + args[1] - y_arr))

def least_squares_rational(args):
    return np.sum(np.square(args[0] / (1 + args[1] * x_arr) - y_arr))

def f(X):
    return 9 - X[0] ** 2 - X[1] ** 2

def gradient(func, args, eps = 1e-5):
    
    a_arg, b_arg = args[0], args[1]

    part_deriv_a = misc.derivative(lambda a: func([a, b_arg]), a_arg, dx = eps)
    part_deriv_b = misc.derivative(lambda b: func([a_arg, b]), b_arg, dx = eps)

    return np.array([part_deriv_a, part_deriv_b])

def gradient_num(func, args, eps = 1e-5):
    
    a_arg, b_arg = args[0], args[1]
    
    #n_digits = abs(int(round(np.log10(eps))))
    #part_deriv_a = round((func([a_arg + eps, b_arg]) - func([a_arg - eps, b_arg])) / (2 * eps), n_digits)
    #part_deriv_b = round((func([a_arg, b_arg + eps]) - func([a_arg, b_arg - eps])) / (2 * eps), n_digits)
    
    part_deriv_a = (func([a_arg + eps, b_arg]) - func([a_arg - eps, b_arg])) / (2 * eps)
    part_deriv_b = (func([a_arg, b_arg + eps]) - func([a_arg, b_arg - eps])) / (2 * eps)

    return np.array([part_deriv_a,  part_deriv_b])


x_arr = np.load('x_arr.npy')
y_arr = np.load('y_arr.npy')

tmp_1 = gradient(least_squares_rational, [1, 0])
tmp_2 = gradient_num(least_squares_rational, [1, 0])
print(tmp_1, tmp_2)

tmp_1 = gradient(least_squares_linear, [1, 0])
tmp_2 = gradient_num(least_squares_linear, [1, 0])
print(tmp_1, tmp_2)

# неужели это ожно и то же???

[-38.44961147  26.96584518] [-38.44961147  26.96584518]
[ -60.29584519 -139.44961147] [ -60.29584519 -139.44961147]
